<a href="https://colab.research.google.com/github/coldbilberry/repo-gui/blob/main/Computer_Vision_%D0%B4%D0%BB%D1%8F_%D0%BC%D0%B5%D0%B4%D0%B8%D1%86%D0%B8%D0%BD%D1%81%D0%BA%D0%B8%D1%85_%D0%B8%D0%B7%D0%BE%D0%B1%D1%80%D0%B0%D0%B6%D0%B5%D0%BD%D0%B8%D0%B9_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задание 2.

-Прочитать статью ""An Improved VGG16 Model for Pneumonia Image Classification"" by Zhi-Peng Jiang et al., 2021.

-Реализовать модель диагностики пневмоторакса на основе IVGG13 (см. статью)

-Выполнить тюнинг гиперпараметров модели и улучшить метрики accuracy, precision, recall по сравнению с исходной моделью, представленной в уроке.

-Провести сравнительный анализ ROC-кривых полученных моделей. Выбрать оптимальный threshold.

In [1]:
pip install --upgrade tensorflow

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [4]:
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [5]:
import seaborn as sns

In [6]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

Загрузка данных

In [ ]:
# Путь к данным
base_dir = "drive/MyDrive/CV для мед изображений/chest_xray"

In [ ]:
# Каталоги
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

In [ ]:
# Создание датасетов
batch_size = 32
img_size = (224, 224)

train_dataset = image_dataset_from_directory(
    train_dir,
    label_mode='binary',
    color_mode='grayscale',
    batch_size=batch_size,
    image_size=img_size,
    shuffle=True
)

val_dataset = image_dataset_from_directory(
    val_dir,
    label_mode='binary',
    color_mode='grayscale',
    batch_size=batch_size,
    image_size=img_size,
    shuffle=True
)

test_dataset = image_dataset_from_directory(
    test_dir,
    label_mode='binary',
    color_mode='grayscale',
    batch_size=batch_size,
    image_size=img_size,
    shuffle=False
)

Модель VGG13

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, InputLayer

def build_vgg13(input_shape):
    model = Sequential([
        InputLayer(input_shape=input_shape),

        # Первый блок Conv + MaxPooling
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2), strides=(2, 2)),

        # Второй блок Conv + MaxPooling
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2), strides=(2, 2)),

        # Третий блок Conv + MaxPooling
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2), strides=(2, 2)),

        # Четвертый блок Conv + MaxPooling
        Conv2D(512, (3, 3), activation='relu', padding='same'),
        Conv2D(512, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2), strides=(2, 2)),

        # Пятый блок Conv + MaxPooling
        Conv2D(512, (3, 3), activation='relu', padding='same'),
        Conv2D(512, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2), strides=(2, 2)),

        # Полносвязные слои
        Flatten(),
        Dense(4096, activation='relu'),
        Dense(4096, activation='relu'),

        # Выходной слой для бинарной классификации
        Dense(1, activation='sigmoid')
    ])

    return model

input_shape = (224, 224, 1)
model = build_vgg13(input_shape)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Обучение

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10
)

Оценка эффективности модели

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

# Получение предсказаний для тестового набора
test_dataset = test_dataset.unbatch()
test_images = []
test_labels = []

In [ ]:
for image, label in test_dataset:
    test_images.append(image.numpy())
    test_labels.append(label.numpy())

In [ ]:
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [ ]:
preds = model.predict(test_images)
y_true = test_labels
y_pred = preds.ravel()

In [ ]:
y_pred_classes = np.where(y_pred > 0.5, 1, 0)

In [ ]:
# Расчет метрик
accuracy = accuracy_score(y_true, y_pred_class)
precision = precision_score(y_true, y_pred_class)
recall = recall_score(y_true, y_pred_class)
f1 = f1_score(y_true, y_pred_class)

In [ ]:
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-score: {f1}')

In [ ]:
# ROC-кривая и AUC
fpr, tpr, thresholds = roc_curve(y_true, y_pred)
auc = roc_auc_score(y_true, y_pred)

In [ ]:
plt.figure()
plt.plot(fpr, tpr, label=f'ROC curve (area = {auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Определение оптимального порога
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
print(f'Optimal threshold: {optimal_threshold}')

In [ ]:
# Пример использования порога
y_pred_class = (y_pred >= optimal_threshold).astype(int)

Ковариационная матрица

In [ ]:
conf_matrix = confusion_matrix(y_true, y_pred_class)
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Normal', 'Pneumothorax'], yticklabels=['Normal', 'Pneumothorax'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

Вывод:

Была реализована модель диагностики пневмоторакса на основе IVGG13. Accurasy получилась довольно высокой = 0.8108.

ROC curve покрывает 0.88 верных предсказаний. Это высокий показатель эффективности модели.

Оптимальный порог (threshold) 0.9999.

По ковариационной матрице: Высокий показатель правильных предсказаний пневмоторокса и нормы. Но есть ошибки предсказаний. Ошибка второго рода, где больному предсказывают отсутствие заболевания = 82. Ошибка первого рода = 36. Необходимо уменьшать ошибку второго рода.

In [ ]:
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-score: {f1}')